In [1]:
import tensorflow as tf

from network.cnn_definition_paper import rgb255_to_obj_net 
from tensorflow.keras.layers import Lambda

from xnp.xnp_tf_layers import MtMCreation, MtMEvaluation
from reverse_op import PODestarisation

from data_handling.model_info import load_model_info
from data_handling import DataLoader

bop_path  = '/export/jesse/BOP'
dataset = 'tless'
dataset_path = f'{bop_path}/{dataset}'

results_path = '../results/'

foreign_info = 'scene_maskrcnn_detections_syn+real_info.json'

Importing Jupyter notebook from /home/jesse/bop/csl/xnp/xnp_numpy.ipynb


In [3]:
def totuplething(x1, x2, x4, x7):
    return ((x1, x2, x4, x7), x7)

for oiu in range(1,31):
    model_info = load_model_info(dataset_path, oiu, verbose=1)

    trained_cnn = tf.keras.models.load_model(f'{dataset_path}/saved_models/csl_o{oiu}_trainable_layers')

    rgb_input, depth_input, cam_K, coord_K = trained_cnn.inputs
    star_representation, dash_representation, w_px, w_d, seg = trained_cnn.outputs

    segmentation = Lambda(lambda x: tf.cast(x>0.75, x.dtype))(seg)
    w_px = Lambda(lambda x: x[0][...,tf.newaxis,:] * x[1][...,tf.newaxis])([w_px, segmentation])
    w_d = Lambda(lambda x: x[0][...,tf.newaxis,:] * x[1][...,tf.newaxis])([w_d, segmentation])

    po_image = PODestarisation(model_info,amount_of_instances = 1)(star_representation, dash_representation, segmentation)
    po_image = Lambda(lambda x: x[0][...,tf.newaxis,:] * x[1][...,tf.newaxis])([po_image, segmentation])
    MtM = MtMCreation("po", strides)((po_image, w_px, cam_K, coord_K))
    pred_T_, pred_Omega = MtMEvaluation()(MtM)

    depth = Lambda(lambda x: x[:,::strides,::strides])(depth_input)
    MtM_d = MtMCreation("depth")((po_image, depth , w_d))
    pred_T_d, pred_Omega = MtMEvaluation()(tf.keras.layers.Add()([MtM, MtM_d]))

    pred_model = tf.keras.Model(trained_cnn.inputs, (pred_T_, pred_T_d))

    test_foreign_data = DataLoader.load_foreign_data([f'{dataset_path}/{d}' for d in test ], foreign_info, oiu)
    print(f'Found TEST foreign data for {len(test_foreign_data)} occurencies of object {oiu}, where {len([d for d in test_foreign_data if "primesense" in d["root"]])} origined from primesense.')

    res = pred_model.predict(DataLoader.Dataset(test_foreign_data,xyDim, test_mode=True).batch(1).prefetch(20).map(totuplething), verbose=1)

    rowwisestr = 'scene_id,im_id,obj_id,score,R,t,time\n'
    rowwisestr_d = 'scene_id,im_id,obj_id,score,R,t,time\n'
    for tdi, td in enumerate(test_foreign_data):
        pose_ = res[0][tdi][0]
        pose_d = res[1][tdi][0]
        rowwisestr += f'{int(td["root"][-6:])},{int(td["file_name"])},{oiu},0.5,{" ".join(list(pose_[:3,:3].flatten().astype(str)))},{" ".join(list(pose_[:3,3].flatten().astype(str)))},-1\n'
        rowwisestr_d += f'{int(td["root"][-6:])},{int(td["file_name"])},{oiu},0.5,{" ".join(list(pose_d[:3,:3].flatten().astype(str)))},{" ".join(list(pose_d[:3,3].flatten().astype(str)))},-1\n'
    
    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb_tless-test.csv","w")
    f.write(rowwisestr)
    f.close()

    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb-d_tless-test.csv","w")
    f.write(rowwisestr_d)
    f.close()

model_info for object 1:
diameter : 63.5151
mins : [-17.4958 -17.4958 -30.6   ]
maxs : [17.4958 17.4958 30.6   ]
symmetries_discrete : []
symmetries_continuous : True
destarring as symmetries_continuous

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change 

100%|██████████| 20/20 [00:00<00:00, 662.24it/s]
/home/jesse/python-virtual-environments/ud_ope/lib/python3.6/site-packages/tensorflow_addons/image/transform_ops.py:143: UserWarning: fill_value is not supported and is always 0 for TensorFlow < 2.4.0.
  "fill_value is not supported and is always 0 for TensorFlow < 2.4.0."


Found TEST foreign data for 1041 occurencies of object 1, where 1041 origined from primesense.
destarring as symmetries_continuous
1041/1041 [==============================] - 64s 62ms/step
model_info for object 2:
diameter : 66.1512
mins : [-21.6448 -21.6448 -30.8511]
maxs : [21.6448 21.6448 30.8511]
symmetries_discrete : []
symmetries_continuous : True
destarring as symmetries_continuous

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is lik

100%|██████████| 20/20 [00:00<00:00, 581.30it/s]


Found TEST foreign data for 1362 occurencies of object 2, where 1362 origined from primesense.
destarring as symmetries_continuous
1362/1362 [==============================] - 85s 62ms/step
model_info for object 3:
diameter : 65.3491
mins : [-23.8837 -23.8842 -30.8351]
maxs : [23.879  23.8774 30.8351]
symmetries_discrete : []
symmetries_continuous : True
destarring as symmetries_continuous

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is lik

...

Found TEST foreign data for 320 occurencies of object 28, where 320 origined from primesense.
destarring as symmetries_discrete with z_factor= 2
po_ was corrected by [-0. -0. -0.]
320/320 [==============================] - 21s 64ms/step
model_info for object 29:
diameter : 134.227
mins : [-56.5 -39.  -28.4]
maxs : [56.5 39.  28.4]
symmetries_discrete : [array([[-1.00000e+00, -4.63077e-14,  0.00000e+00,  0.00000e+00],
       [ 4.63077e-14, -1.00000e+00,  0.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  1.00000e+00,  0.00000e+00],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00]])]
symmetries_continuous : False
destarring as symmetries_discrete with z_factor= 2
generate ref samples discrete symmetries with z_factor= 2
Tensor("BatchGatherND/Reshape_5:0", shape=(None, 15, 3, 8, 3), dtype=float32)
po_ was corrected by [-0. -0. -0.]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an iss

100%|██████████| 20/20 [00:00<00:00, 816.21it/s]


Found TEST foreign data for 203 occurencies of object 29, where 203 origined from primesense.
destarring as symmetries_discrete with z_factor= 2
po_ was corrected by [-0. -0. -0.]
203/203 [==============================] - 13s 65ms/step
model_info for object 30:
diameter : 88.7538
mins : [-40.  -40.  -25.3]
maxs : [40.  40.  25.3]
symmetries_discrete : []
symmetries_continuous : True
destarring as symmetries_continuous

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If

100%|██████████| 20/20 [00:00<00:00, 790.94it/s]


Found TEST foreign data for 273 occurencies of object 30, where 273 origined from primesense.
destarring as symmetries_continuous
273/273 [==============================] - 17s 62ms/step


In [5]:
combined_results = 'scene_id,im_id,obj_id,score,R,t,time\n'
for oiu in range(1,31):
    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb_tless-test.csv","r")
    combined_results += f.read()[len('scene_id,im_id,obj_id,score,R,t,time\n'):]
    f.close()
    
f = open(f"{results_path}/new-paper-1-30-cosypose-frame-rgb_tless-test.csv","w")
f.write(combined_results)
f.close()

combined_results = 'scene_id,im_id,obj_id,score,R,t,time\n'
for oiu in range(1,31):
    f = open(f"{results_path}/new-paper-{oiu}-cosypose-frame-rgb-d_tless-test.csv","r")
    combined_results += f.read()[len('scene_id,im_id,obj_id,score,R,t,time\n'):]
    f.close()
    
f = open(f"{results_path}/new-paper-1-30-cosypose-frame-rgb-d_tless-test.csv","w")
f.write(combined_results)
f.close()
        